# LEIA-Solo Quickstart

**SUMMARY**: *In this lab, you will learn how to setup and connect to LEIA through the `smartleia_target` SDK. We will also cover how to capture power traces, and how to communicate with target devices*

**LEARNING OUTCOMES:**
* Setting up LEIA Hardware
* Using the smartleia Python API to connect to your hardware
* Communication with the target using the smartleia_target SDK
* Capturing a power trace

## Prerequisites

Hold up - before continuing, ensure you have done the following:

* ☑ Run through the Jupyter introduction.

## Physical Setup

### Board pinout

![LEIA Solo Pinout](https://h2lab.org/images/devices/leia_pinout.png "LEIA Pinout")


### Testing points

![LEIA Solo test points TOP](https://h2lab.org/images/devices/leia_test_points_top.png)

![LEIA Solo test points BOT](https://h2lab.org/images/devices/leia_test_points_bot.png)

### Setting up measure Mode

In order to setup LEIA in the measure mode: 

 * Move the PRG1, PRG2, PRG3 (LEIA Solo < v1.4) and PRG4 to the LEIA position (1-2).
 * Remove the shunt bypass jumber if it is set.
 * Move the tearing jumper to the OFF position.
 * Setup the power source for the smartcard. We would advise an external "clean" power source for clean measurements. However, we are able to get proper traces with the USB-C power supply on the funcard. 
 

## Communication with the target using the smart leia

First you need to import the `smartleia` Python module.

In [ ]:
from smartleia import APDU, TriggerPoints

### Configuring the scope

This completely depends on the scope you use. See the scope documentation, usualy this can be done using SCPI commands through ethernet or USB interface

### Configuring the target
Now you can setup the LEIA target with the following:

In [ ]:
target = cw.target(scope, targets.LeiaTarget)
target.init()
wait_for_card(target)
target.configure_smartcard()
target.select_applet()


### Waiting for the card to be inserted

In [ ]:
wait_for_card(target)

### Reseting the target to a know state

In [ ]:
target.reset()

### Select the target AES applet
Once everything is setup, it is possible to select the target applet performing the AES with the `select_applet` method:

In [ ]:
 target.select_applet(applet=[0x45, 0x75, 0x74, 0x77, 0x74, 0x75, 0x36, 0x41, 0x70, 0x80])

### Setting the AES inputs
You can set the AES inputs (key and plaintext) with

In [ ]:
key = "2b7e151628aed2a6abf7158809cf4f3c"
data = "6bc1bee22e409f96e93d7e117393172a"

set_inputs(target, key, data)

### Checking data on the target
You can check that the key and the plaintext has indeed been properly setup in the target using:

In [ ]:
target.checkEncryptionKey(key)
target.checkPlaintext(data)

### Define the trigger strategy
Now, in order to do some trace aquisition campaigns, it is necessary to setup triggers. The LEIA target allows to setup triggers in various points in time when sending an APDU and receiving a response using `set_trigger_strategy`, please refer to the `smartleia` documentation to have more information on this! As shown in the example below, the `TriggerPoints.TRIG_PRE_SEND_APDU` allows to launch a trigger juste before sending the APDU to the smart card, with a `delay` of 0.

In [ ]:
 #Point just before sending a simple APDU
target.set_trigger_strategy(1, point_list=[TriggerPoints.TRIG_PRE_SEND_APDU],delay=0)
print(target.get_trigger_strategy(1))

## Capturing Traces with your favorite scope

Let us start recording some power traces! To capture a trace we will use the `scopy` Python library.
This library enables driving Lecroy, Owon and ChipWhisprer scopes using a simplified common API.

1. Setup you scope manually (nothing to do here if your are using the ChipWhisperer).
1. Arm the scope with `scope.arm()`. It will begin capturing as soon as it is triggered.
1. If you are using a chipwhisperer `scope.capture()` will read back the captured power trace, blocking until either ChipWhisperer is done recording, or the scope times out. Note that the error return status will tell you whether or not the scope timed out. It does not return the captured scope data.
1. Read back the captured power trace with `scope.get_last_trace()`.


In [ ]:
from scopy import *
import smartleia as SL
from smartleia import APDU, TriggerPoints
from smartleia_target import TargetController
import itertools, sys

spinner = itertools.cycle(['-', '/', '|', '\\'])


def reader_wait_for_card(target):
    """
        Wait for the smartcard to be inserted
    """
    print('Waiting for card to be inserted...\t',end='')
    while not(target.is_card_inserted()):
        sys.stdout.write(next(spinner))
        time.sleep(0.5)
        sys.stdout.flush()            
        sys.stdout.write('\b')
    print('OK')


AES_TEST_VECTOR_KEY = "2b7e151628aed2a6abf7158809cf4f3c"
AES_TEST_VECTOR_DATA = "6bc1bee22e409f96e93d7e117393172a"
AES_TEST_VECTOR_CIPHER = "3ad77bb40d7a3660a89ecaf32466ef97"


# selecting your scope and opening the connection through the supported interface
args = {
    'ip': '192.168.100.42',
    'port': 3000,
    'net_tcp': True
}
scope = owon_xds(args)


# initialise target protocol
target = TargetController()
reader_wait_for_card(target)
target.configure_smartcard(
        protocol_to_use = SL.T.T0,
        negotiate_pts = False)

# selecting the target applet
target.select_applet(applet=[0x45, 0x75, 0x74, 0x77, 0x74, 0x75, 0x36, 0x41, 0x70, 0x80])


# Setting trigger Point just before sending a simple APDU
target.set_trigger_strategy(1, point_list=[TriggerPoints.TRIG_PRE_SEND_APDU],delay=0)

# Seting the key
key = AES_TEST_VECTOR_KEY
plaintext = AES_TEST_VECTOR_DATA
target.loadEncryptionKey(key)
target.loadInput(plaintext)

# arm the scope
scope.arm(channels=[1,2]) # We want to capture both the power trace and the trigger signals

# launch computation and get the computation time
timing = target.go().delta_t

# wait for target to finish
while target.isDone() is False and timeout:
    timeout -= -1
    time.sleep(0.01)

try:
    ret = scope.capture()
    if ret:
        print("Timeout happened during acquisition")
except IOError as e:
    print(f"IOError: {e}")

    
cipher = target.readOutput().data

# print the result value
cipherstr = ''.join('{:02x}'.format(x) for x in cipher)

print(cipherstr)

# print the expected value
expectedstr=''.join('{:02x}'.format(x) for x in target.getExpected())

print(expectedstr)
   



## Showing the traces
You can plot the traces with:

In [ ]:
import matplotlib.pyplot as plt

last_trace = scope.get_last_trace()  

plt.plot(last_trace)
plt.show()


## Saving the traces
You can save the traces as `numpy` arrays using:

In [ ]:
import numpy as np
from datetime import datetime
from tqdm import trange

now = datetime.now()
fmt_string = "{:02}{:02}_{}.npy"

traces = []     # captured traces array
datain = []     # testcase data input array
known_keys = [] # testcase key input array

traces.append(last_trace)
datain.append(data)
known_key.append(key)

trace_array = np.asarray(traces)
datain_array = np.asarray(datain)
known_keys = np.asarray(keys)

trace_file_path = os.path.join(workdir, fmt_string.format(now.hour, now.minute, "traces"))
datain_file_path = os.path.join(workdir,fmt_string.format(now.hour, now.minute, "data"))
keys_file_path = os.path.join(workdir,fmt_string.format(now.hour, now.minute, "keys"))

np.save(trace_file_path, trace_array)
np.save(datain_file_path, datain_array)
np.save(keys_file_path, known_keys)
